In [90]:
import numpy as np
import plotly.express.colors
import plotly.graph_objects as go
import notebooks.latex_utils as latex

# P = np.array([ [x, np.sqrt(x)] for x in np.linspace(0, 100, 3)])

P = np.array([
    [3.2, 9.3],
    [3.3, 6.9],
    [1.8, 2.9],
    [1.9, 4.9],
    [4.2, 3.6],
    [8.6, 3.4],
    [10.9, 6.4],
    [10, 10],
    [7.9, 10.8],
    [5.6, 6.5],
    [5.5, 1]
])
# P = np.array([
#     [2.8, 4.5],
#     [1.3, 3.2],
#     [1.4, 1.5],
#     [3, 0.5],
#     [5, 0.8],
#     [5.1, 2.4],
#     [3.5, 3.2],
#     [3, 1.5],
#     [4.2, 2]
# ])
Sx = P[:, 0]
Sy = P[:, 1]
Px = np.array([ [i, p] for i, p in enumerate(Sx)])
Py = np.array([ [i, p] for i, p in enumerate(Sy)])

def interpolate(P, tau):
    n = P.shape[0] - 1

    X = P[:, 0]
    Y = P[:, 1]
    h = np.zeros(n)
    for i in range(n):
        h[i] = X[i + 1] - X[i]

    alpha = np.zeros(n)
    for i in range(n):
        alpha[i] = 1 / h[i] - tau / np.sinh(tau * h[i])

    beta = np.zeros(n)
    for i in range(n):
        beta[i] = tau * (1.0 / np.tanh(tau * h[i])) - 1 / h[i]

    gamma = np.zeros(n)
    for i in range(n):
        gamma[i] = tau ** 2 / h[i] * (Y[i + 1] - Y[i])

    C = np.zeros(n - 1)
    for i in range(n-1):
        C[i] = gamma[i+1] - gamma[i]
    A = np.zeros((n - 1, n-1))

    for i in range(n-1):
        if i == 0:
            A[i, i] = beta[i] + beta[i+1]
        elif i == n-2:
            A[i, i-1] = alpha[n-2]
            A[i, i] = beta[n-2] + beta[n-1]
        else:
            A[i, i-1] = alpha[i]
            A[i, i] = beta[i] + beta[i+1]
            A[i, i+1] = alpha[i+1]

    z = np.concatenate(([0], np.linalg.solve(A, C), [0]))

    tau2 = tau ** 2
    def s(x):
        i = 0
        for j in range(n, -1, -1):
            if x - X[j] >= 0:
                i = j
                break
        if i >= n:
            i = n - 1
        return (z[i] * np.sinh(tau * (X[i + 1] - x)) + z[i + 1] * np.sinh(tau * (x - X[i]))) / (tau2 * np.sinh(tau * h[i])) + \
            (Y[i] - z[i]/tau2) * (X[i+1] - x) / h[i] + (Y[i+1] - z[i+1]/tau2)* (x - X[i])/h[i]
    return s



def scatter(Px, Py, tau, color):
    n = Py.shape[0] - 1
    T = np.linspace(Py[:, 0][0], Py[:, 0][n], 1000)
    Sx = interpolate(Px, tau)
    Sy = interpolate(Py, tau)
    X = [Sx(t) for t in T]
    Y = [Sy(t) for t in T]
    return go.Scatter(x=X, y=Y, line=dict(color=color), mode="lines", name=f"tau={tau}")


figs = [ scatter(Px, Py, tau, color) for color, tau in [('#FFFFFF', 0.1), ('#CAFF70', 5), ('#FFFF00', 100)]]

dataFig = go.Scatter(x=P[:, 0], y=P[:, 1], text=Py[:, 0], line=dict(color='red'), name='Data points', mode="markers+text", textposition='top right')
master = go.Figure(dataFig)
for f in figs:
    master.add_trace(f)

master.update_layout(legend=dict(title_font_family="Times New Roman",font = dict(size=20)))

master.show()
